In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import numpy as np
import pandas as pd
import torch
from PIL import Image  # PIL库的Image包是基于python开发的数字图片处理包。
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms  # 使用torchvision中的变换

# 原文链接：https://blog.csdn.net/beauthy/article/details/124926678

/data1/zhengnanyan/miniconda3/envs/VLM310/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


# 基本流程

## 1 读取png：open

In [4]:
img_id='/data1/zhengnanyan/code/VLM/data/images/菜头.png'
image = Image.open(img_id).convert('RGB')  # [3, 256, 256](通道数,高,宽）= (c, h, w)
imgSize = image.size
imgSize

(485, 444)

## 2 数据集做transform变换

In [6]:
# 训练和验证数据集采用transform做变换
transform = transforms.Compose([
    transforms.Resize([256,256]), # 图片resize
    # transforms.RandomCrop(224),  # 随机裁剪224*224
    transforms.ToTensor(),  # 将图像转为Tensor,数据归一化了欸！img.float().div(255)
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 测试集采用test_transform做变换
test_transform = transforms.Compose([
    transforms.Resize([288, 288]),  # 把图片resize为256*256
    transforms.RandomCrop(256),  # 随机裁剪224*224，测试时无标签
    transforms.RandomHorizontalFlip(),    # 水平翻转
    transforms.ToTensor(),  # 将图像转为Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])   # 标准化
])

## 3 输出tensor结果

In [12]:
image_trans = transform(image)# 返回torch.Size([3, 256, 256])
image_trans

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])

In [10]:
afterSize = image_trans.numpy().shape[1:]  # 缩放后图像的宽高
afterSize

(256, 256)

In [11]:
bi = np.array(afterSize) / np.array(imgSize)
bi

array([0.52783505, 0.57657658])

# 构造dataset